# Sentiment Analysis Model

change current directory and import packages

In [1]:
import os
import numpy as np

In [2]:
if not os.path.exists("/sentiment_analysis"):
    os.chdir("..")

import packages

In [3]:
from sentiment_analysis.utils.train_test_split import TrainTestSplit
from sentiment_analysis.models.model import LightGBMStreamlined
from sentiment_analysis.features.word_frequencies import WordFrequencyVectorizer
from sentiment_analysis.data.review_processor import ReviewProcessor
import lightgbm as lgb

Perform train test split on the reviews data

In [4]:
X_train, y_train, X_test, y_test = TrainTestSplit().get_split_data()

Build lightGBM model and train using the training data

In [5]:
lightgbm = LightGBMStreamlined()
lightgbm.train(X_train, y_train)

get the predictions

In [7]:
X_pred = lightgbm.predict(X_test)

In [14]:
X_pred_train = lightgbm.predict(X_train)

In [18]:
sum(y_train)

2546

X_pred_train and X_pred are all 0s, why? We can have a look with the custom transformer

In [9]:
wf = WordFrequencyVectorizer()
X_train_transformed = wf.transform(X_train)

In [32]:
X_test_transformed = wf.transform(X_test)

In [27]:
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'false',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.06,
    'bagging_fraction': 0.67,
    'bagging_freq': 1,
    'learning_rate': 0.05,
    'verbose_eval': 0,
    'n_jobs': 6
}
lgbm = lgb.LGBMClassifier(**parameters)

Try built in CountVectorizer

In [23]:
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_train)

In [29]:
# X_test = vectorizer.transform(X_test)

In [30]:
lgbm.fit(X.toarray(), y_train)

LGBMClassifier(application='binary', bagging_fraction=0.67, bagging_freq=1,
               boosting='gbdt', feature_fraction=0.06, is_unbalance='false',
               learning_rate=0.05, metric='auc', n_jobs=6, objective='binary',
               verbose_eval=0)

In [31]:
y_pred = lgbm.predict(X_test.toarray())

In [32]:
roc_auc_score(y_test, y_pred)

0.5

We still are only able to get 0.5 AUC, which equivalent to random guessing. Possible reason would be the high level of sparsity existed in the dataset. Most of the reviews are less than 20 words, while the dimensionality of the features after count vectorizer would be at least 20000. Therefore, most of the values in a single vector will be 0s, potentially making lightGBM very hard to fit.

In [ ]:
model = lgb.LGBMClassifier
params = 